In [174]:
import numpy as np
from PIL import Image
import re
from src.css.css_parser import *
from fpdf import FPDF
from src.css.color_utils import *
import os.path
from src.fpdf_extention import *
from src.md_to_pdf_converter import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
colors = get_theme_from_file()
pool = import_obsidian_styles("Obsidian Nord.css")

In [176]:
pool = StylePool()
pool.doc_style.set_background_color(colors["--background-primary"])
pool.style_normal.set_font_color(colors["--text-normal"])

pool.style_h1.set_font_color(colors["--text-title-h1"])
pool.style_h2.set_font_color(colors["--text-title-h2"])
pool.style_h3.set_font_color(colors["--text-title-h3"])
pool.style_h4.set_font_color(colors["--text-title-h4"])
pool.style_h5.set_font_color(colors["--text-title-h5"])
pool.style_h6.set_font_color(colors["--text-title-h6"])

pool.style_code.set_font_color(colors["--code-block"])
pool.style_code.set_background_color(colors["--pre-code"])

pool.style_mark.set_font_color(colors["--text-normal"])
pool.style_mark.set_background_color(colors["--pre-code"])

pool.style_link.set_font_color(colors["--text-accent"])


In [264]:
file_text = read_contents("MDtest.md")

In [266]:
pdf = PrintablePDF()
convert_md_to_pdf(file_text, pdf, pool)
#pdf.link(10,10,100,100, 'http|://www.intranet.com/mb/rprh06/final.php?folio=')

pdf.output("mygfg.pdf")

''